In [1]:
%load_ext autoreload
%autoreload 2

import time

import torch
import torch.optim as optim
from torch.utils.data import DataLoader

import numpy as np
import pickle

from classes.dataset import *
from classes.model import *
from classes.loss import *
from utils import *

In [2]:
size = '3m'
X_encoding = 'bin_y'
y_encoding = 'dense_y'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_data = torch.tensor(np.load(f'data/{X_encoding}_{size}.npy'), device=device)
y_data = torch.tensor(np.load(f'data/{y_encoding}_{size}.npy'), device=device)

idx_data = torch.tensor(np.load(f'data/remove_idx_3m.npy'), device=device)

In [3]:
data_size = X_data.shape[0]

train_dataset = SimpleDynamicDataset(
    X_data[:int(0.8 * data_size)],
    idx_data[:int(0.8 * data_size)],
    y_data[:int(0.8 * data_size)]
)
test_dataset = SimpleDynamicDataset(
    X_data[int(0.8 * data_size):],
    idx_data[int(0.8 * data_size):],
    y_data[int(0.8 * data_size):]
)

train_loader, test_loader = DataLoader(train_dataset, batch_size=100, shuffle=True), DataLoader(test_dataset, batch_size=1000)

del X_data, y_data

In [4]:
epochs = 10

train_steps = len(train_loader)
test_steps = len(test_loader)

print_step = 10

model = RecurrentModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.1)
loss_fn = DoubleLoss()

for epoch in range(epochs):
    print(f'\nEPOCH {epoch + 1}:')
    train_loss = 0.

    start = time.time()
    for i, (X, idx, y) in enumerate(train_loader):
        optimizer.zero_grad()

        cell_pred, update_pred = model(X)

        loss = loss_fn(cell_pred, update_pred, idx, y)

        train_loss += loss.item()

        loss.backward()
        optimizer.step()

        if i % print_step == 0 and i != 0:
            end = time.time()
            
            batch_speed = print_step / (end - start)
            progress = int((i / train_steps) * 50)
            bar = "\u2588" * progress + '-' * (50 - progress)
            
            print(f'Training: |{bar}| {2 * progress}% - loss {train_loss / print_step:.2g} - speed {batch_speed:.2f} batch/s', end='\r', flush=True)
            
            last_batch_speed = batch_speed
            last_train_loss = train_loss
            train_loss = 0.

            start = time.time()

    last_bar = "\u2588" * 50
    print(f'Training: |{last_bar}| {100}% - loss {last_train_loss / print_step:.2g} - speed {last_batch_speed:.2f} batch/s')

    test_loss = 0.
    with torch.no_grad(): # evaluate model on test data
        for X, y in test_loader:
            pred = model(X)

            loss = loss_fn(pred, y)
            test_loss += loss.item()
        
        len_adjust = len(f" |{last_bar}| {100}% - ") - 2
        print(f'Validation:{" " * len_adjust}loss {test_loss / test_steps:.2g}')

    optimizer.param_groups[0]['lr'] *= 0.5 # reduce learning rate by 50% every epoch


EPOCH 1:


: 

In [5]:
one_hot = pickle.load(open('data/encoder.pkl', 'rb'))

for X, y in train_loader:
    break

In [14]:
model(X[idx])*9

tensor([5.1883, 5.3701, 4.5219, 5.4408, 5.0962, 4.9958, 4.5360, 4.9219, 4.7344,
        5.1800, 4.5912, 4.8392, 4.7961, 4.8590, 5.1376, 5.1451, 5.8179, 4.4268,
        5.0523, 5.2717, 4.8538, 4.9465, 4.9096, 4.6401, 5.1326, 4.8765, 5.1721,
        5.1014, 5.2556, 5.1570, 4.9368, 5.5323, 4.6234, 4.9187, 4.8639, 4.5263,
        4.8298, 5.2472, 5.1257, 4.9888, 4.5554, 4.5784, 5.1911, 4.8241, 5.4953,
        4.7937, 4.6837, 4.6994, 5.4996, 5.2515, 5.0040, 5.3597, 4.6713, 4.9314,
        4.5459, 4.8193, 5.5581, 4.6231, 4.8405, 5.1019, 4.9411, 4.9328, 5.4751,
        4.7843, 4.4553, 5.2079, 4.7787, 5.2122, 5.0002, 4.9010, 5.5214, 5.0217,
        5.3894, 5.2437, 4.8377, 4.9595, 4.6708, 5.7859, 4.7467, 4.2857, 5.0627],
       grad_fn=<MulBackward0>)

In [25]:
idx = 0 #np.random.randint(0, 10)

print_grid(X[idx]*9-1, data_type='dense')
print(loss_fn(X[idx], y[idx]))

print_grid((model(X[idx])*9-1).detach(), data_type='dense')
print(loss_fn(model(X[idx].reshape(1, -1)), y[idx].reshape(1, -1)))

print_grid(y[idx]*9-1, data_type='dense')
print(loss_fn(torch.tensor(one_hot.transform(y[idx].reshape(-1, 1)+1).toarray())*100, y[idx]))

-------------------------------
| 1  0  0 | 7  9  0 | 8  6  0 |
| 8  9  5 | 6  4  1 | 0  2  7 |
| 0  3  7 | 5  0  8 | 4  0  0 |
-------------------------------
| 4  7  0 | 1  3  5 | 9  8  6 |
| 9  0  3 | 8  6  0 | 5  0  4 |
| 0  6  8 | 9  0  0 | 2  3  1 |
-------------------------------
| 3  0  0 | 2  0  6 | 7  5  8 |
| 7  5  0 | 3  0  9 | 1  4  0 |
| 0  8  1 | 4  5  7 | 0  9  0 |
-------------------------------
tensor(0.0890)
-------------------------------
| 5  4  5 | 5  5  4 | 5  4  5 |
| 4  4  5 | 5  5  5 | 5  4  4 |
| 5  4  5 | 4  4  5 | 4  5  5 |
-------------------------------
| 5  4  4 | 4  5  4 | 5  5  5 |
| 4  5  4 | 4  5  5 | 4  5  4 |
| 5  4  5 | 5  5  4 | 5  5  4 |
-------------------------------
| 4  5  5 | 5  5  4 | 4  4  4 |
| 4  5  5 | 4  4  4 | 5  5  5 |
| 5  4  4 | 4  5  5 | 4  4  5 |
-------------------------------
tensor(0.0824, grad_fn=<MseLossBackward0>)
-------------------------------
| 1  2  4 | 7  9  3 | 8  6  5 |
| 8  9  5 | 6  4  1 | 3  2  7 |
| 6  3  7 | 5 

/home/juluis/anaconda3/envs/AISudoku/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([81])) that is different to the input size (torch.Size([81, 9])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (9) must match the size of tensor b (81) at non-singleton dimension 1

In [28]:
test_loss = nn.CrossEntropyLoss()

print(test_loss(torch.tensor([[0, 0, 19.]]), torch.tensor([2])))

tensor(0.)


In [18]:
model(X[0].reshape(1, -1)).reshape(81, 10).argmax(dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0])

In [19]:
print_grid(model(X[0].reshape(1, -1)).reshape(81, 10).argmax(dim=1).numpy())

-------------------------------
| 0  0  0 | 0  0  0 | 0  0  0 |
| 0  0  0 | 0  0  0 | 0  0  0 |
| 0  0  0 | 0  0  0 | 7  0  0 |
-------------------------------
| 0  0  0 | 0  0  0 | 0  0  0 |
| 0  0  0 | 0  0  0 | 0  0  0 |
| 0  0  0 | 0  0  0 | 0  0  0 |
-------------------------------
| 0  0  0 | 0  0  0 | 0  0  0 |
| 0  0  0 | 0  0  0 | 0  0  0 |
| 0  0  0 | 0  0  0 | 0  0  0 |
-------------------------------


In [7]:
model(X[0])

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)